In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from python_speech_features import mfcc,logfbank
import librosa

In [ ]:
def plot_signal(signals):
    fig,axes = plt.subplots(nrows=2,ncols=5,sharex=False,sharey=True,figsize=(20,5))
    fig.suptitle('Time series',size=16)
    i=0
    for x in range(2):
        for y in range(5):
            axes[x,y].set_title(list(signals.keys())[i])
            axes[x,y].plot(list(signals.values())[i])
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i+=1

In [ ]:
def plot_fft(fft):
    fig,axes = plt.subplots(nrows=2,ncols=5,sharex=False,
                           sharey=True,figsize=(20,5))
    fig.suptitle('Fourier Transform',size=16)
    i=0
    for x in range(2):
        for y in range(5):
            data= list(fft.values())[i]
            Y,freq =data[0],data[1]
            axes[x,y].set_title(list(fft.keys())[i])
            axes[x,y].plot(freq,Y)
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i+=1

In [ ]:
def plot_fbank(fbank):
    fig,axes = plt.subplots(nrows=2,ncols=5,sharex=False,
                           sharey=True,figsize=(20,5))
    fig.suptitle('Filter Bank Coefficient',size=16)
    i=0
    for x in range(2):
        for y in range(5):
            data= list(fft.values())[i]
            Y,freq =data[0],data[1]
            axes[x,y].set_title(list(fbank.keys())[i])
            axes[x,y].imshow(list(fbank.values())[i],
                            cmap='hot',interpolation='nearest')
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i+=1

In [ ]:
def plot_mfccs(mfccs):
    fig,axes = plt.subplots(nrows=2,ncols=5,sharex=False,
                           sharey=True,figsize=(20,5))
    fig.suptitle('Mel Frequency Cepstrum Coefficients',size=16)
    i=0
    for x in range(2):
        for y in range(5):
            data= list(fft.values())[i]
            Y,freq =data[0],data[1]
            axes[x,y].set_title(list(mfccs.keys())[i])
            axes[x,y].imshow(list(mfccs.values())[i],
                            cmap='hot',interpolation='nearest')
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i+=1

In [ ]:
df = pd.read_csv('../input/freesound-audio-tagging/train.csv')
df

In [ ]:
df.set_index('fname',inplace=True)

In [ ]:
for f in df.index:
    rate,signal = wavfile.read('../input/freesound-audio-tagging/audio_train/'+f)
    df.at[f,'length'] = signal.shape[0]/rate

In [ ]:
df

In [ ]:
classes = list(np.unique(df.label))
class_dist = df.groupby(['label'])['length'].mean()

In [ ]:
classes[0:5]

In [ ]:
class_dist[0:5]

In [ ]:
fig, ax=plt.subplots()
ax.set_title('Class Distribution',y=1.08)
ax.pie(class_dist,labels=class_dist.index,autopct='%1.1f%%;',
      shadow=False,startangle=90)
ax.axis('equal')
plt.show()
df.reset_index(inplace=True)

In [ ]:
def calc_fft(y,rate):
    n = len(y)
    freq= np.fft.rfftfreq(n, d=1/rate)
    Y = abs(np.fft.rfft(y)/n)
    return (Y, freq)

In [ ]:
signals={}
fft={}
fbank={}
mfccs={}

for c in classes:
    wav_file = df[df.label==c].iloc[0,0]
    signal,rate = librosa.load('../input/freesound-audio-tagging/audio_train/'+wav_file,sr=44100)
    signals[c] = signal
    fft[c] = calc_fft(signal,rate)
    
    bank= logfbank(signal[:rate],rate,nfilt=26,nfft=1103).T
    fbank[c] =bank
    mel = mfcc(signal[:rate],rate,numcep=13,nfilt=26,nfft=1103).T
    mfccs[c]=mel


In [ ]:
plot_signal(signals)
plt.show()

In [ ]:
plot_fft(fft)
plt.show()

In [ ]:
plot_fbank(fbank)
plt.show

In [ ]:
plot_mfccs(mfccs)
plt.show()

In [ ]:
def envelope(y,rate,threshold):
    mask=[]
    y= pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10),min_periods=1, center=True).mean()
    for mean in y_mean:
        if mean >threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

In [ ]:
signals={}
fft={}
fbank={}
mfccs={}

for c in classes:
    wav_file = df[df.label==c].iloc[0,0]
    signal,rate = librosa.load('../input/freesound-audio-tagging/audio_train/'+wav_file,sr=44100)
    mask = envelope(signal,rate,0.0005)
    signal = signal[mask]
    signals[c] = signal
    fft[c] = calc_fft(signal,rate)
    
    bank= logfbank(signal[:rate],rate,nfilt=26,nfft=1103).T
    fbank[c] =bank
    mel = mfcc(signal[:rate],rate,numcep=13,nfilt=26,nfft=1103).T
    mfccs[c]=mel

plot_signal(signals)
plt.show()

plot_fft(fft)
plt.show()

plot_fbank(fbank)
plt.show

plot_mfccs(mfccs)
plt.show()